In [133]:
import gymnasium as gym
from gymnasium.spaces import Discrete
from gymnasium.spaces import Box
import numpy as np
import pandas as pd

In [134]:
class TrainingEnv(gym.Env):

    def __init__(self, data, episode_length = 250, budget=10000):
        self.portfolio_value = budget
        self.cur_row_num = 0
        self.starting_row_num = 0
        self.asset_allocation = 0.0
        self.data = data
        self.episode_length = episode_length
    
        # action space: Sell 25%, sell 10%, no change, buy 10%, buy 25% (percentages are of total portfolio value, asset + cash, at each timestep)
        self.action_space = Discrete(5)

        # observation space: Close, Volume, SMA Ratio, RSI, Bandwidth, Asset Allocation
        self.observation_space = Box(low=np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
            high=np.array([np.inf, np.inf, np.inf, 100.0, np.inf, 1.0]), dtype=np.float64)

    # returns the current row in dataframe with current asset allocation appended
    def _get_obs(self):
        obs = np.array(self.data.iloc[self.cur_row_num, :])
        obs = np.append(obs, self.asset_allocation)
        return obs

    # returns current portfolio value
    def _get_info(self):
        return {'Portfolio Value': self.portfolio_value}

    # sets the starting row and starting asset allocation
    def reset(self, seed=5):
        super().reset(seed=seed)

        self.starting_row_num = self.np_random.integers(0, 0.7 * len(self.data) - self.episode_length - 1)

        self.cur_row_num = self.starting_row_num

        rand = self.np_random.random()
        if rand < 0.7:
            self.asset_allocation = 0.0
        else:
            self.asset_allocation = rand
        
        return self._get_obs(), self._get_info()

    # moves to the next row in data, updates reward and current portfolio value
    def step(self, action):
        self.cur_row_num += 1
        if (self.cur_row_num - self.starting_row_num) > self.episode_length:
            terminated = True
        else:
            terminated = False
        truncated = False
        self.asset_allocation = self._action_to_allocation(action)
        obs = self._get_obs()
        rew = self._get_reward()
        info = self._get_info()
        return obs, rew, terminated, truncated, info
    
    # converts action to asset allocation value
    def _action_to_allocation(self, action):
        allocation_change = 0.0
        if action == 0: allocation_change = -.25
        elif action == 1: allocation_change = -.1
        elif action == 2: allocation_change = 0.0
        elif action == 3: allocation_change = .1
        else: allocation_change = 0.25
        return max(0.0, min(1.0, self.asset_allocation + allocation_change))
    
    # returns reward in the form of regular percent return of the total portfolio (stock + cash) over this timestep
    # need to also account for slippage and commission costs - add later
    def _get_reward(self):
        asset_change = (self.data.iloc[self.cur_row_num, 0] - self.data.iloc[self.cur_row_num - 1, 0]) / self.data.iloc[self.cur_row_num - 1, 0]
        new_portfolio_value = self.portfolio_value * (self.asset_allocation * (1.0 + asset_change) + (1.0 - self.asset_allocation))
        reward = (new_portfolio_value - self.portfolio_value) / self.portfolio_value
        self.portfolio_value = new_portfolio_value
        return reward * 100


In [135]:
from stable_baselines3.common.env_checker import check_env

In [136]:
data = pd.read_csv('Amazon Data.csv')
data.drop(labels=['Date'], axis=1, inplace=True)
data

,Close,Volume,SMA Ratio,RSI,Bandwidth
0,0.200260,56136000,1.368536,62.081040,0.451564
1,0.187500,79008000,1.345808,53.301818,0.412582
2,0.195052,75744000,1.323756,42.939429,0.407098
3,0.200000,16296000,1.308761,42.105261,0.347698
4,0.198958,30936000,1.283152,46.601793,0.310476
...,...,...,...,...,...
6851,219.389999,24819700,1.070065,50.749226,0.086124
6852,220.220001,33956600,1.061704,44.291792,0.074616
6853,224.190002,27515600,1.059974,44.360545,0.068819
6854,227.610001,31849800,1.057722,43.023099,0.066103


In [143]:
training_data.describe()

,Close,Volume,SMA Ratio,RSI,Bandwidth
count,4799.000000,4799.000000,4799.000000,4799.000000,4799.000000
mean,0.169364,0.074033,0.361483,0.462516,0.169360
std,0.207661,0.074536,0.109443,0.158764,0.140356
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.038080,0.035333,0.306916,0.351053,0.072242
50%,0.076391,0.054561,0.362009,0.459076,0.124078
75%,0.247831,0.083794,0.409188,0.569186,0.221932
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [144]:
env = TrainingEnv(training_data)
check_env(env, warn=True)

In [145]:
from stable_baselines3 import DQN

env = TrainingEnv(training_data)

model = DQN("MlpPolicy", env)
model.learn(total_timesteps=100)

In [146]:
obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print(info)
    if terminated or truncated:
        obs, info = env.reset()

{'Portfolio Value': 12685.704561319884}
{'Portfolio Value': 12645.08814488027}
{'Portfolio Value': 12750.290037360346}
{'Portfolio Value': 12801.933362473022}
{'Portfolio Value': 12777.2913937794}
{'Portfolio Value': 12763.964950974796}
{'Portfolio Value': 12683.817781796837}
{'Portfolio Value': 12563.801502857676}
{'Portfolio Value': 12127.603429649149}
{'Portfolio Value': 12086.712003397679}
{'Portfolio Value': 12040.085470663236}
{'Portfolio Value': 12111.186074064275}
{'Portfolio Value': 12091.75313444441}
{'Portfolio Value': 12149.180089874595}
{'Portfolio Value': 12078.418949524208}
{'Portfolio Value': 12120.790266763717}
{'Portfolio Value': 12229.665737648818}
{'Portfolio Value': 12218.040445111112}
{'Portfolio Value': 12222.27676841796}
{'Portfolio Value': 12127.032548800518}
{'Portfolio Value': 12173.355779659823}
{'Portfolio Value': 12085.76745191523}
{'Portfolio Value': 12134.630127686736}
{'Portfolio Value': 12098.042908802992}
{'Portfolio Value': 12036.285456160027}
{'Port

KeyboardInterrupt: 